In [1]:
import pandas as pd
pd.set_option("display.max_rows", 500)

gene_list = pd.read_table("./salmon_tx2gene.tsv", header = None)

In [2]:
gene_list = gene_list.loc[:,1].drop_duplicates().to_list()

In [3]:
class solver: 
    def __init__(self, gene_list):
        self.no_let = set()
        fivelet_genes = []
        for gene in gene_list:
            if len(gene) <= 5:
                fivelet_genes.append(gene)
        split_genes = []
        for gene in fivelet_genes:
            split_genes.append(list(gene))
        split_genes = pd.DataFrame(split_genes)
        split_genes = split_genes.fillna(" ")
        self.split_genes = split_genes
        self.possibilities = split_genes
        self.guesses = []

    def __call__(self):
        return self.possibilities

    def guess(self, word, yellow = [], green = []):
        self.guesses.append((word, yellow, green))
        wordlist = list(word)
        for position in yellow:
            pypos = position - 1
            self.possibilities = self.possibilities.loc[~self.possibilities[pypos].isin([wordlist[pypos]]),:]
            self.possibilities = self.possibilities.loc[self.possibilities.isin([wordlist[pypos]]).any(axis=1),:]
            wordlist.pop(pypos)
        for position in green:
            pypos = position - 1
            print(pypos)
            self.possibilities = self.possibilities.loc[self.possibilities[pypos].isin([wordlist[pypos]]),:]
            wordlist.pop(pypos)
        for letter in wordlist:
            self.no_let.add(letter)
        for letter in self.no_let:
            self.possibilities = self.possibilities.loc[~self.possibilities.isin(self.no_let).any(axis=1),:]
        return self.possibilities 
    
    def redo(self):
        for guess_set in self.guesses:
            self.guess(*guess_set)



In [6]:
pd.options.display.max_rows = 500
mysolver = solver(gene_list)
mysolver()

,0,1,2,3,4
0,O,R,4,F,5
1,O,R,4,F,3
2,N,O,C,2,L
3,P,E,R,M,1
4,H,E,S,4,
...,...,...,...,...,...
10884,D,A,Z,3,
10885,D,A,Z,2,
10886,T,T,T,Y,3
10887,C,D,Y,1,B


In [ ]:
mysolver.guess("GAPDH", yellow=[], green=[2])

In [ ]:
mysolver.guess("WARS2", yellow=[], green=[2,5])